# Plotting Path Density

In this notebook we make a plot to compare the result of observed transitions with the instanton + het orbit path


In [ ]:
import xarray as xr
import sys 
sys.path.append('/rds/general/user/cfn18/home/Double-Well-SR/Calculating-Transition-Rate/')
from calculating_transition_times import transition_file_list
import os
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pickle

import seaborn as sns
cp = sns.color_palette("tab10")
cp

## Getting stochastic integration transition data

In [ ]:
# Getting x and y info

x_dict = {}
y_dict = {}
eps = 0.1
sample_size = 1000 #atm we have like 3000 transition max

for a in alphas:
    xs = []
    ys = []
    transition_files = transition_file_list(a, eps, c2h=True, cluster=True)

    for file in tqdm(transition_files[:1000]):
        da = xr.open_dataset(file)
        xs.append(da.x.values)
        x_dict[a] = xs
        ys.append(da.y.values)
        y_dict[a] = ys
        da.close()



## Get MAM Data and Het orbits

In [ ]:
# MAM Results

instantons = {}
wd = '../Minimisation/Data/varying-alpha/cold-to-saddle/'
for s in os.listdir(wd):
    with open(wd + s, 'rb') as file:
        print(f'Opening {wd + s}')
        mam_dict = pickle.load(file)
        instantons[mam_dict['alpha']] = mam_dict['instanton']

# Heteroclinic Orbits
alphas = [0.0, 0.25, 0.5, 1.0]
het_orbits = {}
for a in alphas:
    alpha_str = str(a).replace('.', '_')
    file_name = f'../Deterministic-Model/Data/alpha{alpha_str}/hot-relaxation.nc' 
    het_orbit = xr.open_dataset(file_name) 
    het_orbits[a] = het_orbit

## Doing the plot

In [ ]:
# Function for a streamfield background

def streamfield_background(alpha, ax): 
    # Plot Misc
    ax.set_xlim((-1.5, 1.5))
    ax.set_ylim((-1, 1))
    ax.set_xlabel('x')
    ax.set_ylabel('y')


    # alpha label
    ax.text(-1.2, 0.8, fr'$\alpha = {alpha:.2f}$', fontsize=15, bbox={'facecolor': '1', 'pad': 10}) #

    # Gradient Arrows
    x = np.linspace(-1.5, 1.5, 100)
    y = np.linspace(-1, 1, 100)
    X, Y = np.meshgrid(x, y)
    vx = X*(X**2 - 1) - 2 * alpha * Y 
    vy = alpha *X*(X**2 - 1) + 2 * Y 
    speed = np.sqrt(vx**2 + vy**2)
    ax.streamplot(x, y, -vx, -vy, color='0')
    # ax.pcolormesh(X, Y, speed, cmap = 'Blues') 

    # Fixed Points
    cold_point = plt.Circle((-1, 0), 0.1, color='b', alpha=0.5)
    mid_point = plt.Circle((0, 0), 0.1, color='g', alpha=0.5)
    hot_point = plt.Circle((1, 0), 0.1, color='r', alpha=0.5)
    ax.add_patch(cold_point)
#     ax.add_patch(mid_point)
    ax.add_patch(hot_point)
    return ax

In [ ]:
# Code where the plot is realised
aspect = 3/4
size = 8
fig, axes = plt.subplots(1, 2, figsize = (size, aspect * size))

for a, ax in zip([0.0, 1.0], axes.flatten()):
    instanton = instantons[a]
    het_orbit = het_orbits[a]
    ax.plot(instanton[:, 0], instanton[:, 1], color=cp[3], lw=3, label='Minimiser')
    ax.plot(het_orbit.x, het_orbit.y, '--', color=cp[3], lw=3, label='Heteroclinic Orbit')
    ax.legend()
    streamfield_background(a, ax)
    
    h = ax.hist2d(np.hstack(x_dict[a]), np.hstack(y_dict[a]), range= [[-1.5, 1.5], [-1, 1]], bins=100, density=True,
               cmap=mpl.cm.Blues, cmin=0.1, norm=mpl.colors.LogNorm())
    fig.colorbar(h, cax=ax)


In [ ]:
fig.savefig('Path-Density-Verification.pdf')